In [1]:
# Local CSR 1000v (running under vagrant)
HOST = '127.0.0.1'
PORT_NC = 2223
PORT_SSH = 2222
USER = 'vagrant'
PASS = 'vagrant'

In [3]:
import paramiko

#
# prompts are hostname-dependent
#
PROMPT1 = "csr#"
PROMPT2 = "csr(config)#"
PROMPT3 = "csr(config-if)#"

def get_reply(chan, eom):
    bytes = u''
    while bytes.find(eom)==-1:
        bytes += chan.recv(65535).decode('utf-8')
    return bytes

try:
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    
    client.connect(HOST,
                   port=PORT_SSH,
                   username=USER,
                   password=PASS,
                   look_for_keys=False, allow_agent=False)

    conn = client.invoke_shell()
    print(get_reply(conn, PROMPT1))
    conn.send("show ip int brief\n")
    print(get_reply(conn, PROMPT1))
    conn.send("conf t\n")
    print(get_reply(conn, PROMPT2))
    conn.send("interface lo101\n")
    print(get_reply(conn, PROMPT3))
    conn.send("ip address 172.16.0.1 255.255.255.255\n")
    print(get_reply(conn, PROMPT3))
    conn.send("end\n")
    print(get_reply(conn, PROMPT1))

finally:
    client.close()




csr#
show ip int brief
Interface              IP-Address      OK? Method Status                Protocol
GigabitEthernet1       10.0.2.15       YES DHCP   up                    up      
GigabitEthernet2       unassigned      YES NVRAM  administratively down down    
GigabitEthernet3       unassigned      YES NVRAM  administratively down down    
GigabitEthernet4       unassigned      YES NVRAM  administratively down down    
Loopback101            172.16.0.1      YES manual up                    up      
csr#
conf t
Enter configuration commands, one per line.  End with CNTL/Z.
csr(config)#
interface lo101
csr(config-if)#
ip address 172.16.0.1 255.255.255.255
csr(config-if)#
end
csr#
